In [ ]:
import pandas as pd
import gzip
import json
import ast
import polars as pl
from datetime import datetime
import torch, itertools, os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
import pandas as pd
preview = pd.read_json(r"C:\Users\saarthak\Desktop\ResearcP\Electronics.jsonl.gz", lines=True, nrows=5)
print(preview.head())

   rating                                        title  \
0       3            Smells like gasoline! Going back!   
1       1      Didn’t work at all lenses loose/broken.   
2       5                                   Excellent!   
3       5                       Great laptop backpack!   
4       5  Best Headphones in the Fifties price range!   

                                                text  \
0  First & most offensive: they reek of gasoline ...   
1  These didn’t work. Idk if they were damaged in...   
2  I love these. They even come with a carry case...   
3  I was searching for a sturdy backpack for scho...   
4  I've bought these headphones three times becau...   

                                              images        asin parent_asin  \
0  [{'small_image_url': 'https://m.media-amazon.c...  B083NRGZMM  B083NRGZMM   
1                                                 []  B07N69T6TM  B07N69T6TM   
2                                                 []  B01G8JO5F2  B01G8JO5

In [ ]:
import pandas as pd
preview = pd.read_json(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics.jsonl.gz", lines=True, nrows=5)
print(preview.head())

               main_category  \
0            All Electronics   
1            All Electronics   
2                  Computers   
3             AMAZON FASHION   
4  Cell Phones & Accessories   

                                               title  average_rating  \
0             FS-1051 FATSHARK TELEPORTER V3 HEADSET             3.5   
1                      Ce-H22B12-S1 4Kx2K Hdmi 4Port             5.0   
2  Digi-Tatoo Decal Skin Compatible With MacBook ...             4.5   
3  NotoCity Compatible with Vivoactive 4 band 22m...             4.5   
4             Motorola Droid X Essentials Combo Pack             3.8   

   rating_number                                           features  \
0              6                                                 []   
1              1             [UPC: 662774021904, Weight: 0.600 lbs]   
2            246  [WARNING: Please IDENTIFY MODEL NUMBER on the ...   
3            233  [☛NotoCity 22mm band is designed for Vivoactiv...   
4             64  [

Removing columns with Hyperlink and converting file to csv

In [4]:
input_path = r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics.jsonl.gz"
output_path = r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned.csv"

chunksize = 100_000
first_chunk = True
drop_cols = ["images", "videos"]

with gzip.open(input_path, 'rt', encoding='utf-8') as f:
    batch = []
    for i, line in enumerate(f, 1):
        try:
            obj = json.loads(line)
            batch.append(obj)
        except json.JSONDecodeError:
            print(f"Skipping line {i} (bad JSON)")
            continue

        if i % chunksize == 0:
            df = pd.DataFrame(batch)
            if drop_cols:
                df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)
            df.to_csv(output_path, mode='w' if first_chunk else 'a',
                      index=False, header=first_chunk)
            print(f"Written batch {i // chunksize} ({i} rows)")
            first_chunk = False
            batch.clear()

    if batch:
        df = pd.DataFrame(batch)
        if drop_cols:
            df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)
        df.to_csv(output_path, mode='w' if first_chunk else 'a',
                  index=False, header=first_chunk)
        print(f"Final write: total {i} lines.")


Written batch 1 (100000 rows)
Written batch 2 (200000 rows)
Written batch 3 (300000 rows)
Written batch 4 (400000 rows)
Written batch 5 (500000 rows)
Written batch 6 (600000 rows)
Written batch 7 (700000 rows)
Written batch 8 (800000 rows)
Written batch 9 (900000 rows)
Written batch 10 (1000000 rows)
Written batch 11 (1100000 rows)
Written batch 12 (1200000 rows)
Written batch 13 (1300000 rows)
Written batch 14 (1400000 rows)
Written batch 15 (1500000 rows)
Written batch 16 (1600000 rows)
Final write: total 1610012 lines.


In [4]:
df = pd.read_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned.csv")

columns_to_drop = ['bought_together', 'subtitle', 'author']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

df.to_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned1.csv", index=False)

C:\Users\saarthak\AppData\Local\Temp\ipykernel_11204\677886419.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned.csv")


Renaming columns to avoid clash and confusion with column names

In [ ]:
df = pd.read_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned1.csv")
df.rename(columns={'average_rating': 'product_average_rating','rating_number':'product_rating_number'}, inplace=True)
df.to_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned.csv", index=False)
print("Column renamed and saved as meta_Electronics_cleaned.csv")

C:\Users\saarthak\AppData\Local\Temp\ipykernel_11664\3633288061.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned1.csv")


Column renamed and saved as meta_renamed.csv


In [ ]:
import polars as pl
df = pl.scan_csv(r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned.csv")

df = df.rename({
    "rating": "review_rating",
    "title": "review_title",
    "text": "review_text"
})

df.collect().write_csv(r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned1.csv")


In [2]:
review_df = pl.read_csv(r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned.csv")
review_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
f64,str,str,str,str,str,str,i64,bool
3.0,"""smells like gasoline! going ba…","""first & most offensive: they r…","""B083NRGZMM""","""B083NRGZMM""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""","""2022-07-18T22:58:37.948""",0,true
1.0,"""didn’t work at all lenses loos…","""these didn’t work. idk if they…","""B07N69T6TM""","""B07N69T6TM""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""","""2020-06-20T18:42:29.731""",0,true
5.0,"""excellent!""","""i love these. they even come w…","""B01G8JO5F2""","""B01G8JO5F2""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""","""2018-04-07T09:23:37.534""",0,true
5.0,"""great laptop backpack!""","""i was searching for a sturdy b…","""B001OC5JKY""","""B001OC5JKY""","""AGGZ357AO26RQZVRLGU4D4N52DZQ""","""2010-11-20T18:41:35.000""",18,true
5.0,"""best headphones in the fifties…","""i've bought these headphones t…","""B013J7WUGC""","""B07CJYMRWM""","""AG2L7H23R5LLKDKLBEF2Q3L2MVDA""","""2023-02-17T02:39:41.238""",0,true


In [ ]:
review_df = review_df.with_columns([
    pl.col("review_title").str.to_lowercase().alias("review_title"),
    pl.col("review_text").str.to_lowercase().alias("review_text")
])
review_df.write_csv(r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned.csv")
review_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
f64,str,str,str,str,str,i64,i64,bool
3.0,"""smells like gasoline! going ba…","""first & most offensive: they r…","""B083NRGZMM""","""B083NRGZMM""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",1658185117948,0,true
1.0,"""didn’t work at all lenses loos…","""these didn’t work. idk if they…","""B07N69T6TM""","""B07N69T6TM""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",1592678549731,0,true
5.0,"""excellent!""","""i love these. they even come w…","""B01G8JO5F2""","""B01G8JO5F2""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",1523093017534,0,true
5.0,"""great laptop backpack!""","""i was searching for a sturdy b…","""B001OC5JKY""","""B001OC5JKY""","""AGGZ357AO26RQZVRLGU4D4N52DZQ""",1290278495000,18,true
5.0,"""best headphones in the fifties…","""i've bought these headphones t…","""B013J7WUGC""","""B07CJYMRWM""","""AG2L7H23R5LLKDKLBEF2Q3L2MVDA""",1676601581238,0,true


In [4]:
review_df = review_df.with_columns([
    pl.col("timestamp").cast(pl.String).cast(pl.Datetime("ms")).alias("timestamp")
])
review_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
f64,str,str,str,str,str,datetime[ms],i64,bool
3.0,"""smells like gasoline! going ba…","""first & most offensive: they r…","""B083NRGZMM""","""B083NRGZMM""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",2022-07-18 22:58:37.948,0,true
1.0,"""didn’t work at all lenses loos…","""these didn’t work. idk if they…","""B07N69T6TM""","""B07N69T6TM""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",2020-06-20 18:42:29.731,0,true
5.0,"""excellent!""","""i love these. they even come w…","""B01G8JO5F2""","""B01G8JO5F2""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",2018-04-07 09:23:37.534,0,true
5.0,"""great laptop backpack!""","""i was searching for a sturdy b…","""B001OC5JKY""","""B001OC5JKY""","""AGGZ357AO26RQZVRLGU4D4N52DZQ""",2010-11-20 18:41:35,18,true
5.0,"""best headphones in the fifties…","""i've bought these headphones t…","""B013J7WUGC""","""B07CJYMRWM""","""AG2L7H23R5LLKDKLBEF2Q3L2MVDA""",2023-02-17 02:39:41.238,0,true


In [11]:
review_df.write_csv(r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned1.csv")

In [12]:
review_df.shape

(43884409, 9)

In [1]:
import polars as pl

meta_path = r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned.csv"
meta_df = pl.read_csv(
    meta_path,
    infer_schema_length=10000,                
    null_values=["", "—", "-", "NA", "null"], 
    ignore_errors=True                
)

meta_df.head()

main_category,product_title,product_average_rating,product_rating_number,features,description,price,store,categories,details,parent_asin
str,str,f64,i64,str,str,f64,str,str,str,str
"""All Electronics""","""FS-1051 FATSHARK TELEPORTER V3…",3.5,6,"""[]""","""['Teleporter V3 The “Teleporte…",null,"""Fat Shark""","""['Electronics', 'Television & …","""{'Date First Available': 'Augu…","""B00MCW7G9M"""
"""All Electronics""","""Ce-H22B12-S1 4Kx2K Hdmi 4Port""",5.0,1,"""['UPC: 662774021904', 'Weight:…","""['HDMI In - HDMI Out']""",null,"""SIIG""","""['Electronics', 'Television & …","""{'Product Dimensions': '0.83 x…","""B00YT6XQSE"""
"""Computers""","""Digi-Tatoo Decal Skin Compatib…",4.5,246,"""['WARNING: Please IDENTIFY MOD…","""[]""",19.99,"""Digi-Tatoo""","""['Electronics', 'Computers & A…","""{'Brand': 'Digi-Tatoo', 'Color…","""B07SM135LS"""
"""AMAZON FASHION""","""NotoCity Compatible with Vivoa…",4.5,233,"""['☛NotoCity 22mm band is desig…","""[]""",9.99,"""NotoCity""","""['Electronics', 'Wearable Tech…","""{'Date First Available': 'May …","""B089CNGZCW"""
"""Cell Phones & Accessories""","""Motorola Droid X Essentials Co…",3.8,64,"""['New Droid X Essentials Combo…","""['all Genuine High Quality Mot…",14.99,"""Verizon""","""['Electronics', 'Computers & A…","""{'Product Dimensions': '11.6 x…","""B004E2Z88O"""


In [2]:
meta_df.null_count()

main_category,product_title,product_average_rating,product_rating_number,features,description,price,store,categories,details,parent_asin
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,996839,0,0,0,0


In [3]:
meta_df.shape

(1494961, 11)

In [19]:
meta_df = meta_df.filter(
    (pl.col("product_title").is_not_null()) & (pl.col("product_title").str.strip_chars() != "") &
    (pl.col("main_category").is_not_null()) & (pl.col("main_category").str.strip_chars() != "") &
    (pl.col("store").is_not_null()) & (pl.col("store").str.strip_chars() != "")
)

Extracting column names from the column named 'details'

In [ ]:
import ast

def parse_dict(cell):
    try:
        if cell and cell.strip().startswith("{"):
            return ast.literal_eval(cell)
        return {}
    except Exception:
        return {}

parsed_details = [parse_dict(val) for val in meta_df["details"]]
details_df = pl.from_dicts(parsed_details)
meta_df = meta_df.hstack(details_df)

Dropping useless columns and which have too many null count

In [6]:
meta_df.null_count()

main_category,product_title,product_average_rating,product_rating_number,features,description,price,store,categories,details,parent_asin,Date First Available,Manufacturer,Product Dimensions,Item Weight,Item model number,Is Discontinued By Manufacturer,Brand,Color,Room Type,Material,Theme,Style,Age Range (Description),Reusability,Pattern,Shape,Unit Count,Finish Type,Package Dimensions,Country of Origin,Best Sellers Rank,Other display features,Item Package Dimensions L x W x H,Package Weight,Item Dimensions LxWxH,Brand Name,…,Cooling Method,Maximum Rotational Speed,Batteries Required?,Screen Resolution,Wireless Type,Graphics Card Description,Display Type,Warranty,Frequency Range,Tuner Technology,Water Resistance Level,Wireless Communication Technology,Batteries Included?,Cable Type,Data Transfer Rate,Whats in the box,Controller Type,Recommended Uses For Product,Department,Number of USB 2.0 Ports,Average Battery Life (in hours),Rear Webcam Resolution,CPU Speed,Minimum Focal Length,Output Voltage,Current Rating,Main Power Connector Type,Occasion,Product Care Instructions,Outer Material,Age Range Description,Cache Size,Operation Mode,Operating Voltage,Actuator Type,International Protection Rating,Mechanical Life Expectancy
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,996839,0,0,0,0,85668,214814,764391,267834,600388,858014,398052,902588,1478652,1287719,1474399,1437651,1444803,1465019,1464478,1436921,1390456,1468914,988624,1360027,870775,1389056,1481887,1481708,1329512,1448353,…,1481842,1489274,1463966,1462618,1448485,1465863,1473563,1488805,1488735,1490191,1472251,1468017,1477356,1429338,1469449,1474038,1465433,1417600,1448365,1473218,1479642,1487187,1478744,1485313,1476039,1483034,1485002,1469866,1481720,1494243,1493001,1489617,1491643,1493805,1494440,1492074,1494948


In [30]:
columns_to_drop = [
    'Manufacturer', 'Product Dimensions', 'Item Weight', 'Item model number',
    'Is Discontinued By Manufacturer', 'Brand', 'Color', 'Room Type',
    'Material', 'Theme', 'Style', 'Age Range (Description)', 'Reusability',
    'Pattern', 'Shape', 'Unit Count', 'Finish Type', 'Package Dimensions',
    'Country of Origin', 'Best Sellers Rank', 'Other display features',
    'Item Package Dimensions L x W x H', 'Package Weight',
    'Item Dimensions  LxWxH', 'Brand Name', 'Suggested Users',
    'Number of Items', 'Part Number', 'Model Year', 'Special features',
    'Light Source Type', 'Magnification Maximum', 'Compatible Devices',
    'Objective Lens Description', 'Power Source', 'Compatible Phone Models',
    'Item Dimensions LxWxH', 'Hardware Platform', 'Operating System',
    'Language', 'Format', 'Publication Date', 'Model number',
    'Standing screen display size', 'Form Factor',
    'Memory Storage Capacity', 'Hardware Interface',
    'Connectivity Technology', 'Model Name', 'Speaker Type',
    'Special Feature', 'Number of Channels', 'Media Format Digital Video',
    'Hard Drive', 'Flash Memory Size', 'Hard Drive Interface',
    'Digital Storage Capacity', 'Hard Disk Interface',
    'Hard Disk Form Factor', 'Hard Disk Description', 'Hard Disk Size',
    'Specific Uses For Product', 'Series', 'Hard Drive Rotational Speed',
    'Hard Disk Rotational Speed', 'Surface Recommendation',
    'Number of Pieces', 'Installation Type', 'Max Screen Resolution',
    'Processor', 'RAM', 'Memory Speed', 'Graphics Coprocessor',
    'Chipset Brand', 'Card Description', 'Graphics Card Ram Size',
    'Number of USB 3.0 Ports', 'Processor Brand', 'Number of Processors',
    'Computer Memory Type', 'Optical Drive Type', 'Screen Size', 'CPU Model',
    'Ram Memory Installed Size', 'Lens Type', 'Compatible Mountings',
    'Camera Lens Description', 'Maximum Focal Length', 'Subwoofer Diameter',
    'Mounting Type', 'Surround Sound Channel Configuration', 'Is Waterproof',
    'Film Format Type', 'Film Color', 'Exposure Control Type', 'Motor Type',
    'Signal Format', 'Graphics Ram Size', 'Video Output Interface',
    'Graphics Processor Manufacturer', 'Number of Keys', 'Switch Type',
    'Antenna', 'Resolution', 'Display Technology', 'Orientation',
    'Vehicle Service Type', 'Video Capture Resolution', 'Control Method',
    'Size', 'Shell Type', 'Manufacturer Part Number', 'Display Size',
    'Voice command', 'Human Interface Input', 'Are Batteries Included',
    'Connector Type', 'Number of Ports', 'Item Package Quantity',
    'Flash Memory Type', 'Handle Material', 'Grip Type', 'Connector Gender',
    'Gauge', 'Batteries', 'Total USB Ports', 'Total HDMI Ports', 'Wattage',
    'Power Connector Type', 'Voltage', 'Cooling Method',
    'Maximum Rotational Speed', 'Batteries Required?', 'Screen Resolution',
    'Wireless Type', 'Graphics Card Description', 'Display Type', 'Warranty',
    'Frequency Range', 'Tuner Technology', 'Water Resistance Level',
    'Wireless Communication Technology', 'Batteries Included?', 'Cable Type',
    'Data Transfer Rate', 'Whats in the box', 'Controller Type',
    'Recommended Uses For Product', 'Department', 'Number of USB 2.0 Ports',
    'Average Battery Life (in hours)', 'Rear Webcam Resolution', 'CPU Speed',
    'Minimum Focal Length', 'Output Voltage', 'Current Rating',
    'Main Power Connector Type', 'Occasion', 'Product Care Instructions',
    'Outer Material', 'Age Range Description', 'Cache Size',
    'Operation Mode', 'Operating Voltage', 'Actuator Type',
    'International Protection Rating', 'Mechanical Life Expectancy','details','features',
    'description','price','categories']

meta_df = meta_df.drop([col for col in columns_to_drop if col in meta_df.columns])


In [8]:
meta_df.columns

['main_category',
 'product_title',
 'product_average_rating',
 'product_rating_number',
 'store',
 'categories',
 'parent_asin',
 'Date First Available']

In [31]:
meta_df.head()

main_category,product_title,product_average_rating,product_rating_number,store,parent_asin,Date First Available,sub_category
str,str,f64,i64,str,str,date,str
"""All Electronics""","""FS-1051 FATSHARK TELEPORTER V3…",3.5,6,"""Fat Shark""","""B00MCW7G9M""",2014-08-02,"""Television & Video"""
"""All Electronics""","""Ce-H22B12-S1 4Kx2K Hdmi 4Port""",5.0,1,"""SIIG""","""B00YT6XQSE""",2015-06-03,"""Television & Video"""
"""Computers""","""Digi-Tatoo Decal Skin Compatib…",4.5,246,"""Digi-Tatoo""","""B07SM135LS""",2019-06-03,"""Computers & Accessories"""
"""AMAZON FASHION""","""NotoCity Compatible with Vivoa…",4.5,233,"""NotoCity""","""B089CNGZCW""",2020-05-29,"""Wearable Technology"""
"""Cell Phones & Accessories""","""Motorola Droid X Essentials Co…",3.8,64,"""Verizon""","""B004E2Z88O""",2010-11-26,"""Computers & Accessories"""


In [12]:
meta_df.null_count()

main_category,product_title,product_average_rating,product_rating_number,store,categories,parent_asin,Date First Available
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [15]:
meta_df.shape

(1409293, 8)

In [32]:
meta_df.write_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned1.csv")

In [17]:
meta_df = meta_df.drop_nulls(subset=["Date First Available"])

From cateories column trying to extract subcategory then dropping categories column

In [18]:
meta_df.head()

main_category,product_title,product_average_rating,product_rating_number,store,categories,parent_asin,Date First Available
str,str,f64,i64,str,str,str,date
"""All Electronics""","""FS-1051 FATSHARK TELEPORTER V3…",3.5,6,"""Fat Shark""","""['Electronics', 'Television & …","""B00MCW7G9M""",2014-08-02
"""All Electronics""","""Ce-H22B12-S1 4Kx2K Hdmi 4Port""",5.0,1,"""SIIG""","""['Electronics', 'Television & …","""B00YT6XQSE""",2015-06-03
"""Computers""","""Digi-Tatoo Decal Skin Compatib…",4.5,246,"""Digi-Tatoo""","""['Electronics', 'Computers & A…","""B07SM135LS""",2019-06-03
"""AMAZON FASHION""","""NotoCity Compatible with Vivoa…",4.5,233,"""NotoCity""","""['Electronics', 'Wearable Tech…","""B089CNGZCW""",2020-05-29
"""Cell Phones & Accessories""","""Motorola Droid X Essentials Co…",3.8,64,"""Verizon""","""['Electronics', 'Computers & A…","""B004E2Z88O""",2010-11-26


In [28]:
def extract_subcat(s):
    try:
        lst = ast.literal_eval(s)
        if isinstance(lst, list) and len(lst) > 1:
            return lst[1]
    except Exception:
        pass
    return None

meta_df = meta_df.with_columns([
    pl.col("categories")
      .map_elements(extract_subcat, return_dtype=pl.Utf8)
      .alias("sub_category")
])

print(meta_df.select(["categories", "sub_category"]).head())

shape: (5, 2)
┌─────────────────────────────────┬─────────────────────────┐
│ categories                      ┆ sub_category            │
│ ---                             ┆ ---                     │
│ str                             ┆ str                     │
╞═════════════════════════════════╪═════════════════════════╡
│ ['Electronics', 'Television & … ┆ Television & Video      │
│ ['Electronics', 'Television & … ┆ Television & Video      │
│ ['Electronics', 'Computers & A… ┆ Computers & Accessories │
│ ['Electronics', 'Wearable Tech… ┆ Wearable Technology     │
│ ['Electronics', 'Computers & A… ┆ Computers & Accessories │
└─────────────────────────────────┴─────────────────────────┘


In [12]:
review_df.null_count()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,1,2534,0,0,0,0,0,0


In [ ]:
review_df = review_df.filter(pl.col("review_text").is_not_null())
review_df = review_df.filter(pl.col("review_title").is_not_null())

In [15]:
review_df.null_count()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [ ]:
meta_df.write_csv(r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned1.csv")

In [ ]:
review_df.write_csv(r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned1.csv")

Joined both review and meta file on the basis of parent_asin

In [ ]:
review_path = r"C:\Users\saarthak\Desktop\ResearcP\Electronics_cleaned1.csv"
meta_path   = r"C:\Users\saarthak\Desktop\ResearcP\meta_Electronics_cleaned1.csv"
out_csv     = r"joined_output.csv"

In [2]:
reviews_lazy = pl.scan_csv(review_path)
meta_lazy    = pl.scan_csv(meta_path)

joined_lazy = reviews_lazy.join(
    meta_lazy,
    on="parent_asin",
    how="inner" 
)

joined_lazy.sink_csv(out_csv)

In [ ]:
joined_df=pl.read_csv(r"joined_output.csv")
joined_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category
f64,str,str,str,str,str,str,i64,bool,str,str,f64,i64,str,str,str
5.0,"""cute for photos!""","""thin material but perfect to h…","""B07RJHQSGB""","""B09NHSGWPD""","""AE74DYR3QUGVPZJ3P7RFWBGIX7XQ""","""2021-11-02T16:22:59.357""",0,true,"""Camera & Photo""","""BELECO 7x5ft Fabric Cowboys Ph…",4.7,101,"""BELECO""","""2019-05-04""","""Camera & Photo"""
5.0,"""great cover and protection""","""fits my macbook air perfectly!…","""B06VTQK7CB""","""B06VTQK7CB""","""AHITBJSS7KYUBVZPX7M2WJCOIVKQ""","""2019-06-08T17:16:26.774""",0,true,"""Computers""","""MOSISO Compatible with MacBook…",4.4,1020,"""MOSISO""","""2017-02-13""","""Computers & Accessories"""
4.0,"""asurion 2 year portable electr…","""good to have security just in …","""B07P97SHQP""","""B07P97SHQP""","""AEFKF6R2GUSK2AWPSWRR4ZO36JVQ""","""2020-12-03T06:17:22.921""",0,true,"""All Electronics""","""ASURION 2 Year Portable Electr…",4.5,1807,"""ASURION""","""2019-02-28""","""Electronics Warranties"""
5.0,"""male to male svga cable""","""great item for the price; alre…","""B0031VXJ7G""","""B07CDCX945""","""AEFKF6R2GUSK2AWPSWRR4ZO36JVQ""","""2013-02-07T10:55:53.000""",0,true,"""Computers""","""Cmple - VGA SVGA Cable Gold Pl…",4.4,461,"""Cmple""","""2009-12-24""","""Computers & Accessories"""
5.0,"""great small speaker. good sou…","""this is a great little speaker…","""B01MTB55WH""","""B0B3T4JWN9""","""AHCPZDDPHJE3G7M6ST5WGRPLXHOA""","""2020-07-15T10:30:04.571""",0,true,"""Home Audio & Theater""","""Anker Soundcore 2 12W Portable…",4.6,110144,"""Anker""","""2017-11-24""","""Portable Audio & Video"""


In [3]:
joined_df.null_count()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1087984


In [4]:
joined_df.shape

(38200208, 16)

Dropping Nulls and removing products which have less than 1000 ratings and whose verified purchase returns to be true 

In [5]:
joined_df = joined_df.drop_nulls(subset=["sub_category"])

In [7]:
joined_df = joined_df.filter(
    (pl.col("product_rating_number") > 1000) & (pl.col("verified_purchase") == True))

In [8]:
joined_df = joined_df.with_columns([
    pl.col("timestamp").cast(pl.String).cast(pl.Datetime("ms")).alias("timestamp")
])
joined_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category
f64,str,str,str,str,str,datetime[ms],i64,bool,str,str,f64,i64,str,str,str
5.0,"""great cover and protection""","""fits my macbook air perfectly!…","""B06VTQK7CB""","""B06VTQK7CB""","""AHITBJSS7KYUBVZPX7M2WJCOIVKQ""",2019-06-08 17:16:26.774,0,true,"""Computers""","""MOSISO Compatible with MacBook…",4.4,1020,"""MOSISO""","""2017-02-13""","""Computers & Accessories"""
4.0,"""asurion 2 year portable electr…","""good to have security just in …","""B07P97SHQP""","""B07P97SHQP""","""AEFKF6R2GUSK2AWPSWRR4ZO36JVQ""",2020-12-03 06:17:22.921,0,true,"""All Electronics""","""ASURION 2 Year Portable Electr…",4.5,1807,"""ASURION""","""2019-02-28""","""Electronics Warranties"""
5.0,"""great small speaker. good sou…","""this is a great little speaker…","""B01MTB55WH""","""B0B3T4JWN9""","""AHCPZDDPHJE3G7M6ST5WGRPLXHOA""",2020-07-15 10:30:04.571,0,true,"""Home Audio & Theater""","""Anker Soundcore 2 12W Portable…",4.6,110144,"""Anker""","""2017-11-24""","""Portable Audio & Video"""
5.0,"""down in da boot, it is what ya…","""riding out hurricanes is not s…","""B003A21DQA""","""B00E0ISVLI""","""AH6PLOGWYIVIWLJTY756BHNFD4YA""",2022-02-19 17:14:14.067,1,true,"""Home Audio & Theater""","""Kaito KA500 5-way Powered Emer…",4.6,12083,"""Kaito""","""2004-09-01""","""Portable Audio & Video"""
5.0,"""... in my sewing room and this…","""needed a tv in my sewing room …","""B00KSBB7EY""","""B00KSBB7EY""","""AGCIVNDDRX3PXIUINEG4NCRGWCRA""",2015-03-23 17:12:42,1,true,"""All Electronics""","""Samsung UN24H4500 24-Inch 720p…",4.4,1245,"""SAMSUNG""","""2014-07-13""","""Television & Video"""


In [9]:
joined_df.shape

(18365091, 16)

In [11]:
joined_df = joined_df.sort("timestamp")
joined_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category
f64,str,str,str,str,str,datetime[ms],i64,bool,str,str,f64,i64,str,str,str
5.0,"""please re-release this video!""","""i've seen this movie several t…","""630365455X""","""630365455X""","""AE5KN3WCYFE6CPGILUOU6UD4MNEQ""",1999-11-26 02:41:21,3,true,"""Movies & TV""","""Home for the Holidays VHS""",4.7,3401,"""Holly Hunter (Actor), Rob…","""2006-10-01""","""Camera & Photo"""
5.0,"""how could any1 not love this m…","""hi i am only 12 but i love ros…","""6300215695""","""6300215695""","""AFY7UORBYT47AIH5UR7VOY3Z2KKA""",1999-12-19 15:54:36,1,true,"""Movies & TV""","""White Christmas [VHS]""",4.9,19053,"""Bing Crosby (Actor), Dann…","""2006-12-07""","""Camera & Photo"""
5.0,"""sony mdrv6 headphones - a clas…","""i can't recall how long i've o…","""B00001WRSJ""","""B00WSLYQ7C""","""AGYC42WOF22EOMDKQZ5XWF3XNKFA""",2000-03-15 06:13:32,391,true,"""Home Audio & Theater""","""Sony MDRV6 Studio Monitor Head…",4.6,3738,"""Sony""","""2001-10-02""","""Headphones, Earbuds & Accessor…"
5.0,"""best in the market""","""besides having a built-in swit…","""B00004SB92""","""B00004SB92""","""AEOAYZLLCKHXPHHG5IBJCPB2CZWA""",2000-04-23 12:34:06,11,true,"""All Electronics""","""Cisco-Linksys BEFSR41 EtherFas…",4.0,1070,"""Linksys""","""2003-06-30""","""Computers & Accessories"""
5.0,"""perfect for airplanes""","""different headphones have diff…","""B00001P4XA""","""B00GN7GVCK""","""AEKWJM7QZOGYI3DIBH5KZB3IMMOA""",2000-04-25 20:09:51,10,true,"""Home Audio & Theater""","""Koss 'The Plug' In-Ear Headpho…",4.2,2443,"""Koss""","""2002-07-08""","""Headphones, Earbuds & Accessor…"


In [12]:
helpful_count = joined_df.filter(pl.col("helpful_vote") >= 0).height
print(f"Number of reviews with helpful votes >= 0: {helpful_count}")

Number of reviews with helpful votes >= 0: 18365089


In [13]:
joined_df = joined_df.filter(pl.col("helpful_vote") >= 0)
joined_df.head()

review_rating,review_title,review_text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category
f64,str,str,str,str,str,datetime[ms],i64,bool,str,str,f64,i64,str,str,str
5.0,"""please re-release this video!""","""i've seen this movie several t…","""630365455X""","""630365455X""","""AE5KN3WCYFE6CPGILUOU6UD4MNEQ""",1999-11-26 02:41:21,3,true,"""Movies & TV""","""Home for the Holidays VHS""",4.7,3401,"""Holly Hunter (Actor), Rob…","""2006-10-01""","""Camera & Photo"""
5.0,"""how could any1 not love this m…","""hi i am only 12 but i love ros…","""6300215695""","""6300215695""","""AFY7UORBYT47AIH5UR7VOY3Z2KKA""",1999-12-19 15:54:36,1,true,"""Movies & TV""","""White Christmas [VHS]""",4.9,19053,"""Bing Crosby (Actor), Dann…","""2006-12-07""","""Camera & Photo"""
5.0,"""sony mdrv6 headphones - a clas…","""i can't recall how long i've o…","""B00001WRSJ""","""B00WSLYQ7C""","""AGYC42WOF22EOMDKQZ5XWF3XNKFA""",2000-03-15 06:13:32,391,true,"""Home Audio & Theater""","""Sony MDRV6 Studio Monitor Head…",4.6,3738,"""Sony""","""2001-10-02""","""Headphones, Earbuds & Accessor…"
5.0,"""best in the market""","""besides having a built-in swit…","""B00004SB92""","""B00004SB92""","""AEOAYZLLCKHXPHHG5IBJCPB2CZWA""",2000-04-23 12:34:06,11,true,"""All Electronics""","""Cisco-Linksys BEFSR41 EtherFas…",4.0,1070,"""Linksys""","""2003-06-30""","""Computers & Accessories"""
5.0,"""perfect for airplanes""","""different headphones have diff…","""B00001P4XA""","""B00GN7GVCK""","""AEKWJM7QZOGYI3DIBH5KZB3IMMOA""",2000-04-25 20:09:51,10,true,"""Home Audio & Theater""","""Koss 'The Plug' In-Ear Headpho…",4.2,2443,"""Koss""","""2002-07-08""","""Headphones, Earbuds & Accessor…"


In [15]:
joined_df = joined_df.drop(['verified_purchase'])

In [16]:
joined_df.shape

(18365089, 15)

In [18]:
joined_df.write_csv(r"joined_output.csv")

In [ ]:
joinedf = pl.read_csv("joined_output.csv")

In [2]:
joined_df = joinedf.drop(['asin','user_id'])

In [13]:
joined_df.head()

review_rating,review_title,review_text,parent_asin,timestamp,helpful_vote,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category
f64,str,str,str,str,i64,str,str,f64,i64,str,str,str
5.0,"""please re-release this video!""","""i've seen this movie several t…","""630365455X""","""1999-11-26T02:41:21.000""",3,"""Movies & TV""","""Home for the Holidays VHS""",4.7,3401,"""Holly Hunter (Actor), Rob…","""2006-10-01""","""Camera & Photo"""
5.0,"""how could any1 not love this m…","""hi i am only 12 but i love ros…","""6300215695""","""1999-12-19T15:54:36.000""",1,"""Movies & TV""","""White Christmas [VHS]""",4.9,19053,"""Bing Crosby (Actor), Dann…","""2006-12-07""","""Camera & Photo"""
5.0,"""sony mdrv6 headphones - a clas…","""i can't recall how long i've o…","""B00WSLYQ7C""","""2000-03-15T06:13:32.000""",391,"""Home Audio & Theater""","""Sony MDRV6 Studio Monitor Head…",4.6,3738,"""Sony""","""2001-10-02""","""Headphones, Earbuds & Accessor…"
5.0,"""best in the market""","""besides having a built-in swit…","""B00004SB92""","""2000-04-23T12:34:06.000""",11,"""All Electronics""","""Cisco-Linksys BEFSR41 EtherFas…",4.0,1070,"""Linksys""","""2003-06-30""","""Computers & Accessories"""
5.0,"""perfect for airplanes""","""different headphones have diff…","""B00GN7GVCK""","""2000-04-25T20:09:51.000""",10,"""Home Audio & Theater""","""Koss 'The Plug' In-Ear Headpho…",4.2,2443,"""Koss""","""2002-07-08""","""Headphones, Earbuds & Accessor…"


In [ ]:
df = joined_df.with_columns([
    pl.col("timestamp")
      .str.strptime(
          pl.Datetime,
          "%Y-%m-%dT%H:%M:%S.%f"
      )
      .alias("timestamp")
])

cutoff = datetime(2014, 1, 1)
df_filtered = df.filter(pl.col("timestamp") >= cutoff)

print(f"Kept {df_filtered.height} of {df.height} rows")

C:\Users\saarthak\AppData\Local\Temp\ipykernel_9600\319050612.py:3: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  .str.strptime(


Kept 17472787 of 18365089 rows


In [21]:
df_filtered.shape

(17472787, 13)

In [22]:
df_filtered.write_csv(r"joined_output.csv")

Binning on year and month

In [1]:
IN_CSV  = "joined_output.csv"
OUT_CSV = "joined_binned.csv"
BIN     = "month"

def add_bins(lf: pl.LazyFrame, granularity: str) -> pl.LazyFrame:
    lf = lf.with_columns(
            pl.col("timestamp").str.to_datetime().alias("timestamp")
        ).drop_nulls("timestamp")

    if granularity == "month":
        return lf.with_columns([
            pl.col("timestamp").dt.truncate("1mo").alias("time_bin"),
            pl.col("timestamp").dt.year().alias("year"),
            pl.col("timestamp").dt.month().alias("month"),
        ])
    elif granularity == "week":
        return lf.with_columns([
            pl.col("timestamp").dt.truncate("1w").alias("time_bin"),
            pl.col("timestamp").dt.year().alias("year"),
            pl.col("timestamp").dt.week().alias("week"),
        ])
    elif granularity == "quarter":
        return lf.with_columns([
            pl.col("timestamp").dt.truncate("3mo").alias("time_bin"),
            pl.col("timestamp").dt.year().alias("year"),
            ((pl.col("timestamp").dt.month()-1)//3 + 1).alias("quarter"),
        ])
    else:
        raise ValueError("BIN must be 'week', 'month', or 'quarter'")

(
    pl.scan_csv(IN_CSV, infer_schema_length=100, low_memory=True)
      .pipe(add_bins, BIN)
      .sink_csv(OUT_CSV, batch_size=100_000)
)

print("time‑binned CSV →", OUT_CSV)


✅  time‑binned CSV → joined_binned.csv


In [ ]:
joined_df = pl.read_csv('joined_binned.csv')
joined_df.head() 

review_rating,review_title,review_text,parent_asin,timestamp,helpful_vote,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category,time_bin,year,month
f64,str,str,str,str,i64,str,str,f64,i64,str,str,str,str,i64,i64
5.0,"""love it!""","""i have wanted a digital frame …","""B00DYQQSSK""","""2014-01-01T00:00:28.000000""",0,"""Camera & Photo""","""NIX 12 inch Hi-Res Digital Pho…",4.3,1949,"""NIX""","""2013-07-16""","""Camera & Photo""","""2014-01-01T00:00:00.000000""",2014,1
5.0,"""kindle fast charge""","""this little 9w fast charge rea…","""B006GWO5WK""","""2014-01-01T00:00:38.000000""",0,"""Amazon Devices""","""Amazon Kindle 9W PowerFast Ada…",4.5,16780,"""Amazon""","""2012-08-07""","""Computers & Accessories""","""2014-01-01T00:00:00.000000""",2014,1
5.0,"""still reliable""","""just plug it in and don't look…","""B0019EHU8G""","""2014-01-01T00:00:42.000000""",0,"""Home Audio & Theater""","""Mediabridge™ HDMI Cable (6 Fee…",4.6,20201,"""Mediabridge""","""2008-05-23""","""Television & Video""","""2014-01-01T00:00:00.000000""",2014,1
5.0,"""really great!""","""bought for my wife as she love…","""B00CKYVYI6""","""2014-01-01T00:02:09.000000""",1,"""All Electronics""","""G-Project G-Boom Wireless Blue…",4.4,1679,"""G-Project""","""2013-04-29""","""Portable Audio & Video""","""2014-01-01T00:00:00.000000""",2014,1
5.0,"""works just like it should""","""fits a 96 ram 1500 perfect and…","""B07C9X5RCT""","""2014-01-01T00:03:03.000000""",0,"""Car Electronics""","""BOSS Audio Systems BRS69 120 W…",4.2,4145,"""BOSS Audio Systems""","""2008-10-23""","""Car & Vehicle Electronics""","""2014-01-01T00:00:00.000000""",2014,1


Sentment scoring through RoBERTa

In [ ]:
IN_CSV   = "joined_binned.csv"
OUT_CSV  = "sent_scored.csv"

MODEL_ID = "cardiffnlp/twitter-roberta-base-sentiment-latest"
LABEL2SCORE = {0: -1, 1: 0, 2: 1}
CHUNK_ROWS = 200_000     
BATCH = 64     
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tok   = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID).to(DEVICE).eval()

@torch.inference_mode()
def bert_batch(texts):
    enc = tok(texts, padding=True, truncation=True, max_length=128,
              return_tensors="pt").to(DEVICE)
    preds = model(**enc).logits.argmax(-1).cpu().numpy()
    return [LABEL2SCORE[int(p)] for p in preds]

def score_chunk(pdf: pd.DataFrame) -> pd.DataFrame:
    full = (pdf["review_title"].fillna("") + ". " + pdf["review_text"].fillna("")).tolist()

    scores = list(itertools.chain.from_iterable(
        bert_batch(full[i:i+BATCH]) for i in range(0, len(full), BATCH)
    ))
    pdf["bert_score"] = scores
    return pdf

if os.path.exists(OUT_CSV):
    os.remove(OUT_CSV)

for idx, chunk in enumerate(pd.read_csv(IN_CSV, chunksize=CHUNK_ROWS)):
    scored = score_chunk(chunk)

    mode = "w" if idx == 0 else "a"
    header = (idx == 0)
    scored.to_csv(OUT_CSV, mode=mode, index=False, header=header)
    print(f"✓ chunk {idx+1}  rows={len(scored)}")

print("All chunks processed – output at", OUT_CSV)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✓ chunk 1  rows=200000
✓ chunk 2  rows=200000
✓ chunk 3  rows=200000
✓ chunk 4  rows=200000
✓ chunk 5  rows=200000
✓ chunk 6  rows=200000
✓ chunk 7  rows=200000
✓ chunk 8  rows=200000
✓ chunk 9  rows=200000
✓ chunk 10  rows=200000
✓ chunk 11  rows=200000
✓ chunk 12  rows=200000
✓ chunk 13  rows=200000
✓ chunk 14  rows=200000
✓ chunk 15  rows=200000
✓ chunk 16  rows=200000
✓ chunk 17  rows=200000
✓ chunk 18  rows=200000
✓ chunk 19  rows=200000
✓ chunk 20  rows=200000
✓ chunk 21  rows=200000
✓ chunk 22  rows=200000
✓ chunk 23  rows=200000
✓ chunk 24  rows=200000
✓ chunk 25  rows=200000
✓ chunk 26  rows=200000
✓ chunk 27  rows=200000
✓ chunk 28  rows=200000
✓ chunk 29  rows=200000
✓ chunk 30  rows=200000
✓ chunk 31  rows=200000
✓ chunk 32  rows=200000
✓ chunk 33  rows=200000
✓ chunk 34  rows=200000
✓ chunk 35  rows=200000
✓ chunk 36  rows=200000
✓ chunk 37  rows=200000
✓ chunk 38  rows=200000
✓ chunk 39  rows=200000
✓ chunk 40  rows=200000
✓ chunk 41  rows=200000
✓ chunk 42  rows=200000
✓

Filtering data on the basis of year

In [4]:
score_df = pl.read_csv('sent_scored.csv')
score_df.head()

review_rating,review_title,review_text,parent_asin,timestamp,helpful_vote,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category,time_bin,year,month,bert_score
f64,str,str,str,str,i64,str,str,f64,i64,str,str,str,str,i64,i64,i64
5.0,"""love it!""","""i have wanted a digital frame …","""B00DYQQSSK""","""2014-01-01T00:00:28.000000""",0,"""Camera & Photo""","""NIX 12 inch Hi-Res Digital Pho…",4.3,1949,"""NIX""","""2013-07-16""","""Camera & Photo""","""2014-01-01T00:00:00.000000""",2014,1,1
5.0,"""kindle fast charge""","""this little 9w fast charge rea…","""B006GWO5WK""","""2014-01-01T00:00:38.000000""",0,"""Amazon Devices""","""Amazon Kindle 9W PowerFast Ada…",4.5,16780,"""Amazon""","""2012-08-07""","""Computers & Accessories""","""2014-01-01T00:00:00.000000""",2014,1,1
5.0,"""still reliable""","""just plug it in and don't look…","""B0019EHU8G""","""2014-01-01T00:00:42.000000""",0,"""Home Audio & Theater""","""Mediabridge™ HDMI Cable (6 Fee…",4.6,20201,"""Mediabridge""","""2008-05-23""","""Television & Video""","""2014-01-01T00:00:00.000000""",2014,1,1
5.0,"""really great!""","""bought for my wife as she love…","""B00CKYVYI6""","""2014-01-01T00:02:09.000000""",1,"""All Electronics""","""G-Project G-Boom Wireless Blue…",4.4,1679,"""G-Project""","""2013-04-29""","""Portable Audio & Video""","""2014-01-01T00:00:00.000000""",2014,1,1
5.0,"""works just like it should""","""fits a 96 ram 1500 perfect and…","""B07C9X5RCT""","""2014-01-01T00:03:03.000000""",0,"""Car Electronics""","""BOSS Audio Systems BRS69 120 W…",4.2,4145,"""BOSS Audio Systems""","""2008-10-23""","""Car & Vehicle Electronics""","""2014-01-01T00:00:00.000000""",2014,1,1


In [6]:
counts = (
    score_df
    .group_by("bert_score")
    .count()
    .sort("bert_score", descending=True)    
)

print(counts)

shape: (3, 2)
┌────────────┬──────────┐
│ bert_score ┆ count    │
│ ---        ┆ ---      │
│ i64        ┆ u32      │
╞════════════╪══════════╡
│ 1          ┆ 12776520 │
│ 0          ┆ 1271285  │
│ -1         ┆ 3424982  │
└────────────┴──────────┘


C:\Users\saarthak\AppData\Local\Temp\ipykernel_20064\3407887002.py:4: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  .count()


In [1]:
df = pl.read_csv('sent_scored.csv')

In [2]:
df = df.with_columns(
    pl.col("Date First Available").str.strptime(pl.Date, strict=False)
)

In [3]:
filteredf = df.filter(
    pl.col("Date First Available").dt.year() >= 2014
)

In [4]:
filteredf = filteredf.drop('time_bin')

In [5]:
filteredf.head()

review_rating,review_title,review_text,parent_asin,timestamp,helpful_vote,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category,year,month,bert_score
f64,str,str,str,str,i64,str,str,f64,i64,str,date,str,i64,i64,i64
4.0,"""great for traveling""","""for what i am using it for, it…","""B00G3788AS""","""2014-01-01T00:06:33.000000""",2,"""Computers""","""Dell Venue 8 Pro 5000 Series 3…",4.0,2195,"""Dell""",2017-06-28,"""Computers & Accessories""",2014,1,1
5.0,"""perfect""","""works perfectly with bluetooth…","""B077Y93YRR""","""2014-01-01T00:10:46.000000""",0,"""Home Audio & Theater""","""JAM Classic Wireless Bluetooth…",4.2,7537,"""Jam""",2014-05-01,"""Portable Audio & Video""",2014,1,1
5.0,"""working perfect""","""working perfectly with my sams…","""B07SDH2Z56""","""2014-01-01T00:16:07.000000""",0,"""Home Audio & Theater""","""C&E High Speed HDMI Cable with…",4.6,4012,"""C&E""",2015-04-01,"""Television & Video""",2014,1,1
5.0,"""great to have for less than pe…","""my son does a lot of outdoor p…","""B0B11GNM6K""","""2014-01-01T00:16:53.000000""",0,"""Camera & Photo""","""OpTech USA | Rainsleeve Series…",4.4,2627,"""OP/TECH USA""",2015-06-18,"""Camera & Photo""",2014,1,1
4.0,"""hard to get on""","""boy this thing is a tight fit!…","""B00GXUZA4M""","""2014-01-01T00:17:19.000000""",0,"""Home Audio & Theater""","""Bose SoundLink Mini Soft Cover…",4.6,3273,"""Bose""",2015-06-21,"""Portable Audio & Video""",2014,1,1


In [7]:
filteredf = filteredf.drop_nulls()

In [8]:
filteredf.null_count()

review_rating,review_title,review_text,parent_asin,timestamp,helpful_vote,main_category,product_title,product_average_rating,product_rating_number,store,Date First Available,sub_category,year,month,bert_score
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Considering only 3 main categories

In [ ]:
filteredf = filteredf.with_columns([
    pl.col("main_category").str.strip_chars('"').alias("main_category"),
    pl.col("sub_category").str.strip_chars('"').alias("sub_category")
])

all_elec_subs = (
    filteredf
    .filter(pl.col("main_category") == "All Electronics")
    .select("sub_category")
    .unique()
    .get_column("sub_category")
)

updated_df = filteredf.with_columns([
    pl.when(pl.col("sub_category").is_in(all_elec_subs))
      .then(pl.lit("All Electronics"))
      .otherwise(pl.col("main_category"))
      .alias("main_category_updated")
])

updated_df = updated_df.drop("main_category").rename({"main_category_updated": "main_category"})

C:\Users\saarthak\AppData\Local\Temp\ipykernel_21312\258961606.py:20: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  updated_df = filteredf.with_columns([


In [14]:
print("Original count:", filteredf.shape[0])
print("Updated count:", updated_df.shape[0])

Original count: 14265559
Updated count: 14265559


In [11]:
all_elec_subs = (
    updated_df
    .filter(pl.col("main_category") == "All Electronics")
    .select("sub_category")
    .unique()
    .get_column("sub_category")
)

computer_subs = (
    updated_df
    .filter(pl.col("main_category") == "Computers")
    .select("sub_category")
    .unique()
    .filter(~pl.col("sub_category").is_in(all_elec_subs))
    .get_column("sub_category")
)

updated_df = updated_df.with_columns([
    pl.when(
        (pl.col("main_category") != "All Electronics") &
        (pl.col("sub_category").is_in(computer_subs))
    )
    .then(pl.lit("Computers"))
    .otherwise(pl.col("main_category"))
    .alias("main_category")
])


C:\Users\saarthak\AppData\Local\Temp\ipykernel_21312\1312656723.py:16: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  .filter(~pl.col("sub_category").is_in(all_elec_subs))
C:\Users\saarthak\AppData\Local\Temp\ipykernel_21312\1312656723.py:21: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  updated_df = updated_df.with_columns([


In [13]:
used_subs = updated_df.filter(
    pl.col("main_category").is_in(["All Electronics", "Computers"])
).select("sub_category").unique().get_column("sub_category")

cell_phone_subs = (
    updated_df.filter(pl.col("main_category") == "Cell Phones & Accessories")
    .select("sub_category").unique()
    .filter(~pl.col("sub_category").is_in(used_subs))
    .get_column("sub_category")
)

updated_df = updated_df.with_columns([
    pl.when(
        (~pl.col("main_category").is_in(["All Electronics", "Computers"])) &
        (pl.col("sub_category").is_in(cell_phone_subs))
    )
    .then(pl.lit("Cell Phones & Accessories"))
    .otherwise(pl.col("main_category"))
    .alias("main_category")
])


C:\Users\saarthak\AppData\Local\Temp\ipykernel_21312\2759047172.py:10: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  .filter(~pl.col("sub_category").is_in(used_subs))
C:\Users\saarthak\AppData\Local\Temp\ipykernel_21312\2759047172.py:15: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  updated_df = updated_df.with_columns([


In [15]:
final_df = updated_df.filter(
    pl.col("main_category").is_in(["All Electronics", "Computers", "Cell Phones & Accessories"])
)

In [18]:
print(final_df.group_by("main_category").count())
print("Final row count:", final_df.shape[0])

shape: (3, 2)
┌───────────────────────────┬─────────┐
│ main_category             ┆ count   │
│ ---                       ┆ ---     │
│ str                       ┆ u32     │
╞═══════════════════════════╪═════════╡
│ Cell Phones & Accessories ┆ 75212   │
│ All Electronics           ┆ 7379117 │
│ Computers                 ┆ 6791234 │
└───────────────────────────┴─────────┘
Final row count: 14245563


C:\Users\saarthak\AppData\Local\Temp\ipykernel_21312\1567819490.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  print(final_df.group_by("main_category").count())


In [17]:
final_df = final_df.with_columns([
    pl.when(
        (pl.col("main_category") == "All Electronics") &
        (pl.col("sub_category") == "Computers & Accessories")
    )
    .then(pl.lit("Computers"))
    .otherwise(pl.col("main_category"))
    .alias("main_category")
])

In [19]:
review_counts = final_df.group_by(["main_category", "sub_category"]).agg(
    pl.count("review_text").alias("review_text_count")
)

print(review_counts)

shape: (48, 3)
┌───────────────────────────┬─────────────────────────────────┬───────────────────┐
│ main_category             ┆ sub_category                    ┆ review_text_count │
│ ---                       ┆ ---                             ┆ ---               │
│ str                       ┆ str                             ┆ u32               │
╞═══════════════════════════╪═════════════════════════════════╪═══════════════════╡
│ Cell Phones & Accessories ┆ Electronics Gift Guide - Top T… ┆ 49068             │
│ Cell Phones & Accessories ┆ Featured iPad Accessories       ┆ 239               │
│ All Electronics           ┆ Electronics Warranties          ┆ 82545             │
│ All Electronics           ┆ SkullcandyHeadphones            ┆ 23                │
│ All Electronics           ┆ Headphones, Earbuds & Accessor… ┆ 1936517           │
│ …                         ┆ …                               ┆ …                 │
│ Computers                 ┆ PC Gaming & Accessories        

In [20]:
review_counts.write_csv('cou.csv')

In [30]:
low_all_elec = (
    review_counts.filter(
        (pl.col("main_category") == "All Electronics") & 
        (pl.col("review_text_count") < 5000)
    )
    .select("sub_category")
    .to_series()
)

low_computers = (
    review_counts.filter(
        (pl.col("main_category") == "Computers") & 
        (pl.col("review_text_count") < 3350)
    )
    .select("sub_category")
    .to_series()
)

final_df = final_df.filter(
    ~(
        ((pl.col("main_category") == "All Electronics") & pl.col("sub_category").is_in(low_all_elec)) |
        ((pl.col("main_category") == "Computers") & pl.col("sub_category").is_in(low_computers))
    )
)

C:\Users\saarthak\AppData\Local\Temp\ipykernel_20600\1283152737.py:22: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  final_df = final_df.filter(


In [22]:
final_df.write_parquet('2014_data.parquet')